# 3d mitochondria visualizer using vtk
This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


In [6]:
astrocytes = pd.read_csv('data/astrocytes.csv', index_col=[0])
astrocytes.head()

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,88240,60656,256,648518346349517321,glia,astrocyte
1,72368,56912,590,648518346341392909,glia,astrocyte
4,72704,54688,506,648518346349536799,glia,astrocyte
5,74576,75520,1163,648518346349525537,glia,astrocyte
6,79328,67360,445,648518346349525544,glia,astrocyte


In [7]:
astrocyte_list = astrocytes.cell_segid.to_list()
print(astrocyte_list)

[648518346349517321, 648518346341392909, 648518346349536799, 648518346349525537, 648518346349525544, 648518346349526583, 648518346349490239, 648518346349524070, 648518346349386860, 648518346349508722, 648518346349536888, 648518346349516953, 648518346349521062, 648518346342468778, 648518346342795947, 648518346349524139, 648518346342917290, 648518346349528249, 648518346349528250, 648518346349507266, 648518346349498574, 648518346349528271, 648518346349525715, 648518346341399264, 648518346349536487, 648518346349532924, 648518346349527316, 648518346349527317, 648518346349527319, 648518346342807831, 648518346349537050, 648518346349538089, 648518346341356348, 648518346349517141, 648518346349539158, 648518346349525862, 648518346344075119, 648518346341354380, 648518346349528465, 648518346349484948, 648518346342797245, 648518346349521344, 648518346342795202, 648518346349530569]


# Use index value to view an astrocyte from list 

In [8]:
# pull all mitos from a cellid of interest from astrocyte list, by index number

indx = 37
cellid = astrocyte_list[indx]

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf.sort_values(by='mito_vx', ascending=False)


,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
4186,2526416,78158580,93642,61286,1787,86768,54870,1484,99966,66696,2043,648518346341354380,335238.36,219403.88,71480
2761,2755939,17322744,92404,62536,1964,90750,60096,1750,93446,65962,2157,648518346341354380,330806.32,223878.88,78560
3660,2858484,14863032,93544,57392,1695,92382,55678,1641,95528,59786,1814,648518346341354380,334887.52,205463.36,67800
15469,2880893,9943636,93856,66100,1969,93176,64214,1891,94730,69248,2085,648518346341354380,336004.48,236638.00,78760
26668,3092762,9349704,99018,61756,1986,96382,59654,1854,101364,63672,2048,648518346341354380,354484.44,221086.48,79440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84429,2987170,1272,93862,64960,1773,93838,64942,1770,93876,64982,1776,648518346341354380,336025.96,232556.80,70920
83264,2744783,1256,90158,55872,1868,90140,55854,1867,90184,55892,1871,648518346341354380,322765.64,200021.76,74720
82981,3078957,1252,95632,56324,1864,95616,56310,1862,95654,56346,1866,648518346341354380,342362.56,201639.92,74560
81588,2639192,1232,88148,60948,1828,88134,60930,1827,88168,60970,1830,648518346341354380,315569.84,218193.84,73120


In [9]:
cellid

648518346341354380

In [10]:
thresh = 2000000

In [11]:
mito_query_abovethresh = mito_querydf[mito_querydf.mito_vx > thresh]
mito_query_abovethresh

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
325,3094008,7493880,100658,61202,1860,97588,59934,1752,102422,61888,1980,648518346341354380,360355.64,219103.16,74400
2761,2755939,17322744,92404,62536,1964,90750,60096,1750,93446,65962,2157,648518346341354380,330806.32,223878.88,78560
3660,2858484,14863032,93544,57392,1695,92382,55678,1641,95528,59786,1814,648518346341354380,334887.52,205463.36,67800
4186,2526416,78158580,93642,61286,1787,86768,54870,1484,99966,66696,2043,648518346341354380,335238.36,219403.88,71480
5126,2289990,3396208,84846,60866,1846,82850,59552,1784,86548,61546,1935,648518346341354380,303748.68,217900.28,73840
8215,2645108,5787608,89968,62368,2046,89056,61334,1944,91058,62872,2157,648518346341354380,322085.44,223277.44,81840
10880,2871496,3085160,93512,62306,1930,92826,61820,1830,94084,62580,2049,648518346341354380,334772.96,223055.48,77200
15469,2880893,9943636,93856,66100,1969,93176,64214,1891,94730,69248,2085,648518346341354380,336004.48,236638.00,78760
26668,3092762,9349704,99018,61756,1986,96382,59654,1854,101364,63672,2048,648518346341354380,354484.44,221086.48,79440
36823,3078943,2413200,96170,57288,1915,95602,56428,1855,96774,58368,1946,648518346341354380,344288.60,205091.04,76600


In [12]:
mito_query_belowthresh = mito_querydf[mito_querydf.mito_vx <= thresh]
mito_query_belowthresh

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
369,2645322,382336,89414,62878,1616,89324,62584,1609,89506,63134,1626,648518346341354380,320102.12,225103.24,64640
392,3300338,1698756,102004,58992,2066,101374,58498,2045,102880,59598,2084,648518346341354380,365174.32,211191.36,82640
1339,2744200,738340,89748,57166,2002,89626,57030,1977,89862,57330,2029,648518346341354380,321297.84,204654.28,80080
3022,3200649,1102840,98672,63046,1891,97766,62592,1860,99652,63380,1929,648518346341354380,353245.76,225704.68,75640
3306,2644412,60404,88368,63302,2153,88300,63248,2150,88438,63368,2157,648518346341354380,316357.44,226621.16,86120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409325,3425723,1472392,102910,65830,1938,102312,65412,1889,103472,66148,1971,648518346341354380,368417.80,235671.40,77520
2409468,2648799,1744916,88312,64342,1653,87932,63896,1629,88716,64710,1691,648518346341354380,316156.96,230344.36,66120
2409485,3079568,1786308,96482,55894,1987,96176,55638,1929,96782,56478,2024,648518346341354380,345405.56,200100.52,79480
2409519,2629658,1903396,88636,56388,1726,88454,55706,1676,88898,57120,1773,648518346341354380,317316.88,201869.04,69040


In [13]:
mito_query_abovethresh_mitolist = mito_query_abovethresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_abovethresh_mitolist)))
print(f"type: "+str(type(mito_query_abovethresh_mitolist)))
print('')
# print(mito_query_abovethresh_mitolist) # uncomment to print the mito id list

length: 31
type: <class 'list'>



In [14]:
mito_query_belowthresh_mitolist = mito_query_belowthresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_belowthresh_mitolist)))
print(f"type: "+str(type(mito_query_belowthresh_mitolist)))
print('')
# print(mito_query_belowthresh_mitolist) # uncomment to print the mito id list

length: 964
type: <class 'list'>



# vtk 3d viewer

In [15]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [16]:
cell_id = cellid

In [17]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [18]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [19]:
len(mito_query_abovethresh_mitolist)

31

In [20]:
len(mito_query_belowthresh_mitolist)

964

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [21]:
# download the mito meshes for this cell

for i in range(len(mito_query_abovethresh_mitolist)):
    mito_id = mito_query_abovethresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_id = mito_query_belowthresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)


In [22]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    mito_seg_id = mito_query_abovethresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_abovethresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_abovethresh)


var_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_seg_id = mito_query_belowthresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_belowthresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_belowthresh)


In [23]:
len(var_iterator_abovethresh)

31

In [24]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_abovethresh = 1 # 0.35
mito_color_abovethresh = (0.2, 0.4, 0.7)
mito_opac_belowthresh = 1
mito_color_belowthresh = (0.7, 0.2, 0.2)

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_abovethresh['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator_abovethresh['mito_mesh_'+str(i)], opacity=mito_opac_abovethresh, color=(mito_color_abovethresh))
    
locals().update(actor_iterator_abovethresh)

actor_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_belowthresh['mito_mesh_actor_' + str(i+(len(var_iterator_abovethresh)))] = trimesh_vtk.mesh_actor(var_iterator_belowthresh['mito_mesh_'+str(i)], opacity=mito_opac_belowthresh, color=(mito_color_belowthresh))
    
locals().update(actor_iterator_belowthresh)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_abovethresh)
actor_iterator.update(actor_iterator_belowthresh)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=200) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001A102CCAF70) at 0x000001A12665C4C8>

In [25]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

In [26]:
cellid

648518346341354380

In [27]:
# 648518346349536888 has two to three astrocytes

In [28]:
# 648518346341354380 very interesting case

In [29]:
astrocyte_list

[648518346349517321,
 648518346341392909,
 648518346349536799,
 648518346349525537,
 648518346349525544,
 648518346349526583,
 648518346349490239,
 648518346349524070,
 648518346349386860,
 648518346349508722,
 648518346349536888,
 648518346349516953,
 648518346349521062,
 648518346342468778,
 648518346342795947,
 648518346349524139,
 648518346342917290,
 648518346349528249,
 648518346349528250,
 648518346349507266,
 648518346349498574,
 648518346349528271,
 648518346349525715,
 648518346341399264,
 648518346349536487,
 648518346349532924,
 648518346349527316,
 648518346349527317,
 648518346349527319,
 648518346342807831,
 648518346349537050,
 648518346349538089,
 648518346341356348,
 648518346349517141,
 648518346349539158,
 648518346349525862,
 648518346344075119,
 648518346341354380,
 648518346349528465,
 648518346349484948,
 648518346342797245,
 648518346349521344,
 648518346342795202,
 648518346349530569]